In [1]:
%load_ext autoreload
%autoreload 2

from config import *
from dataset import WavDataset

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa 
import os
import h5py
import pickle

from pathlib import Path
from config import *
from maad import sound, util
from maad.rois import template_matching

In [2]:
df = pd.read_csv('../../output/annotations/manual_annotations/initial_manual_annotations.csv')
df

,min_t,max_t,min_f,max_f,label,recording
0,437.526667,437.986667,1768.06,4014.97,nr_syllable_3khz,1_20230821_060000.wav
1,438.280000,438.826667,1910.69,3926.95,nr_syllable_3khz,1_20230821_060000.wav
2,439.033333,439.553333,1565.13,3798.53,nr_syllable_3khz,1_20230821_060000.wav
3,439.853333,440.313333,1385.49,3840.86,nr_syllable_3khz,1_20230821_060000.wav
4,440.646667,441.193333,1432.34,4014.97,nr_syllable_3khz,1_20230821_060000.wav
...,...,...,...,...,...,...
1976,406.308000,407.064000,5964.76,7874.75,fast_trill_6khz,1_20230828_063000.wav
1977,410.489333,411.693333,5964.76,8324.64,fast_trill_6khz,1_20230828_063000.wav
1978,417.769333,418.740000,5964.76,7984.89,fast_trill_6khz,1_20230828_063000.wav
1979,423.388000,424.125333,6132.78,8209.81,fast_trill_6khz,1_20230828_063000.wav


In [3]:
df['duration'] =  df.max_t - df['min_t']
df

,min_t,max_t,min_f,max_f,label,recording,duration
0,437.526667,437.986667,1768.06,4014.97,nr_syllable_3khz,1_20230821_060000.wav,0.460000
1,438.280000,438.826667,1910.69,3926.95,nr_syllable_3khz,1_20230821_060000.wav,0.546667
2,439.033333,439.553333,1565.13,3798.53,nr_syllable_3khz,1_20230821_060000.wav,0.520000
3,439.853333,440.313333,1385.49,3840.86,nr_syllable_3khz,1_20230821_060000.wav,0.460000
4,440.646667,441.193333,1432.34,4014.97,nr_syllable_3khz,1_20230821_060000.wav,0.546667
...,...,...,...,...,...,...,...
1976,406.308000,407.064000,5964.76,7874.75,fast_trill_6khz,1_20230828_063000.wav,0.756000
1977,410.489333,411.693333,5964.76,8324.64,fast_trill_6khz,1_20230828_063000.wav,1.204000
1978,417.769333,418.740000,5964.76,7984.89,fast_trill_6khz,1_20230828_063000.wav,0.970667
1979,423.388000,424.125333,6132.78,8209.81,fast_trill_6khz,1_20230828_063000.wav,0.737333


In [4]:
for label, d in df.groupby('label'):
    print(label, d.duration.mean(), d.duration.max(), d.duration.min())


fast_trill_6khz 1.111492495531184 2.015999999999991 1.041699999859702e-05
nr_syllable_3khz 0.5204588923137258 1.2480000000000473 0.3280000000000882
triangle_3khz 0.7071478713826086 1.3559999999999945 1.041699999859702e-05
upsweep_500hz 0.8348559411230282 1.5840000000000032 0.015999999999991132


In [23]:
filtered_df = df.loc[df.duration > 0.50]


In [24]:
for label, d in filtered_df.groupby('label'):
    print(label, d.duration.mean(), d.duration.max(), d.duration.min())


fast_trill_6khz 1.1208705712342744 2.015999999999991 0.5786666660000037
nr_syllable_3khz 0.5645651672231834 1.2480000000000473 0.5013333329999341
triangle_3khz 0.7529179229547732 1.3559999999999945 0.5013333339999804
upsweep_500hz 0.9118339438351647 1.5840000000000032 0.5040000000000191
